In [26]:
from snowflake.snowpark.session import Session
import logging, sys
import matplotlib.pyplot as plt
from snowflake.snowpark.functions import udf, col
import streamlit as st

sys.path.append('../src/python/lutils')
import sflk_base as L
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from io import BytesIO
import base64, json ,logging
import random

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR='../'

# Initialize a session with Snowflake
config = L.get_config(PROJECT_HOME_DIR)
sp_session = None
if "snowpark_session" not in st.session_state:
    sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)
    sp_session.use_role(f'''{config['APP_DB']['role']}''')
    sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
    sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')
    st.session_state['snowpark_session'] = sp_session
else:
    sp_session = st.session_state['snowpark_session']


2023-05-26 10:48:31.181 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2023-05-26 10:48:31.181 INFO    util: Connecting to snowflake ...
2023-05-26 10:48:31.183 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.16, Platform: macOS-10.16-x86_64-i386-64bit
2023-05-26 10:48:31.184 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2023-05-26 10:48:31.184 INFO    snowflake.connector.connection: Setting use_openssl_only mode to False
2023-05-26 10:48:31.969 INFO    snowflake.snowpark.session: Snowpark Session information: 
"version" : 1.2.0,
"python.version" : 3.8.16,
"python.connector.version" : 2.7.12,
"python.connector.sessi

In [7]:
key_counter=0
data_table = "IMAGES_ENCODED"
data_col = 'DATA'
name_col = 'NAME'
label_col = 'LABEL'
embedding_col = "EMBEDDING"

sdf = sp_session.table(data_table)
img_names = sdf.select(name_col).to_pandas()

embeddings = sdf.select(embedding_col, name_col).to_pandas()
embeddings

2023-05-26 09:18:12.811 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM IMAGES_ENCODED]
2023-05-26 09:18:12.993 INFO    snowflake.connector.cursor: query execution done
2023-05-26 09:18:13.015 INFO    snowflake.connector.cursor: query: [SELECT "NAME" FROM IMAGES_ENCODED]
2023-05-26 09:18:13.359 INFO    snowflake.connector.cursor: query execution done
2023-05-26 09:18:14.133 INFO    snowflake.connector.cursor: query: [SELECT "EMBEDDING", "NAME" FROM IMAGES_ENCODED]
2023-05-26 09:18:14.518 INFO    snowflake.connector.cursor: query execution done


,EMBEDDING,NAME
0,"[0.006, 0.155, 0.894, 0.867, 0.065, 0.131, 0.1...",shorts_741f5fb3-1745-4bb5-8caa-f0e44ad4b1ed.jpg
1,"[1.238, 0.046, 1.454, 0.599, 1.47, 0.763, 1.96...",undershirt_f784db1b-a842-41d0-8e1a-b5ba48a6cf9...
2,"[1.137, 2.818, 0.084, 1.435, 4.067, 0.23, 1.75...",hoodie_f24486da-981c-4611-a96a-ee1d2558453c.jpg
3,"[0.897, 1.057, 1.52, 0.749, 0.006, 3.189, 1.56...",not_sure_8b98a6bd-5fb9-452f-9057-38086fe7843c.jpg
4,"[0.573, 0.698, 1.019, 0.604, 0.672, 0.866, 0.6...",t_shirt_de9a10ff-3e02-48fb-9cd6-6b87540fcd83.jpg
...,...,...
5757,"[0.342, 0.817, 0.086, 1.859, 1.264, 3.435, 0.7...",longsleeve_c0832183-181f-4e80-857e-038136011b0...
5758,"[0.295, 0.132, 0.01, 1.846, 0.06, 1.485, 0.139...",skirt_ee0fde64-aac5-4904-a8df-921864d7c01c.jpg
5759,"[0.939, 3.831, 1.333, 0.406, 2.053, 1.892, 1.0...",shirt_6050dd9d-c65c-4c57-9b07-00328b8aace3.jpg
5760,"[0.704, 2.775, 1.255, 0.619, 1.879, 1.648, 2.2...",outwear_bdd8bca9-5f78-42d8-96ca-9cc1b8eed481.jpg


In [ ]:
embeddings_np = list(embeddings[embedding_col].apply(lambda x: np.array(json.loads(x))))
similarity=cosine_similarity(embeddings_np)

sim_df = pd.DataFrame(similarity, 
                      columns=embeddings.NAME, 
                      index=embeddings.NAME)

In [27]:
import snowflake.snowpark.functions as F


import base64
img = '../data/train/blazer/00b8048d-635e-4e56-b182-071fb24eea32.jpg'

with open(img, "rb") as image_file:
    encoded_img = base64.b64encode(image_file.read()).decode("utf-8")




sdf = sp_session.create_dataframe([[encoded_img]], schema=["DATA"])
scored_sdf = sdf.with_column('EMBEDDING', F.call_udf("udf_onnx_embedding_model", F.col("DATA")))

2023-05-26 10:48:39.139 INFO    snowflake.connector.cursor: query: [SELECT $1 AS "DATA" FROM  VALUES ('/9j/4AAQSkZJRgABAQEASABIAAD/4SUGRXhpZgAATU0AK...]
2023-05-26 10:48:41.872 INFO    snowflake.connector.cursor: query execution done


In [29]:
scored_sdf.collect()

2023-05-26 10:49:23.966 INFO    snowflake.connector.cursor: query: [SELECT "DATA", udf_onnx_embedding_model("DATA") AS "EMBEDDING" FROM ( SELECT $1 ...]
2023-05-26 10:50:37.441 INFO    snowflake.connector.cursor: Force exit



Force exit


SystemExit: 1

In [8]:
help(embedding)

Help on Column in module snowflake.snowpark.column object:

class Column(builtins.object)
 |  Column(expr: Union[str, snowflake.snowpark._internal.analyzer.expression.Expression]) -> None
 |  
 |  Represents a column or an expression in a :class:`DataFrame`.
 |  
 |  To access a Column object that refers a column in a :class:`DataFrame`, you can:
 |  
 |      - Use the column name.
 |      - Use the :func:`functions.col` function.
 |      - Use the :func:`DataFrame.col` method.
 |      - Use the index operator ``[]`` on a dataframe object with a column name.
 |      - Use the dot operator ``.`` on a dataframe object with a column name.
 |  
 |      >>> from snowflake.snowpark.functions import col
 |      >>> df = session.create_dataframe([["John", 1], ["Mike", 11]], schema=["name", "age"])
 |      >>> df.select("name").collect()
 |      [Row(NAME='John'), Row(NAME='Mike')]
 |      >>> df.select(col("name")).collect()
 |      [Row(NAME='John'), Row(NAME='Mike')]
 |      >>> df.select(df